In [2]:
import pandas as pd
import numpy as np
import re

data = pd.read_excel('File Path/TongHop.xlsx', sheet_name=None)
bad = data['DocHai'].drop(columns=['Thời gian', 'thoigian', 'Chi tiết']).rename(columns={'MoTa': 'Info', 'SuKien': 'Type', 'MAC_Nguon': 'MAC_id'})

persistence_bad = bad[bad['Type'] == 'Persistence'].copy()
persistence_bad['Info'] = persistence_bad['Info'].apply(lambda x: x.split('ValueData: ')[1].split('--')[0])

process_bad = bad[bad['Type'] == 'Process'].copy()
process_bad['Info'] = process_bad['Info'].apply(lambda x: x.split('--')[0])

black_bad = bad[bad['Type'] == 'Black Domain'].copy()
black_bad['Info'] = black_bad['Info'].apply(lambda x: x.split('---')[1])

file_bad = bad[bad['Type'] == 'Create File'].copy()
file_bad['Info'] = file_bad['Info'].apply(lambda x: x.split('FileName: ')[1].split('--')[0])

malware_bad = bad[bad['Type'] == 'Malware'].copy()
malware_bad['Info'] = malware_bad['Info'].apply(lambda x: x.split('--')[1].split(';')[0])
malware_bad['Info'] = malware_bad['Info'].apply(lambda x: re.sub(r'[\(].*[\)]', '', x))
malware_bad['Info'] = malware_bad['Info'].apply(lambda x: re.sub(r'\\\s', '', x))
malware_bad['Info'] = malware_bad['Info'].apply(lambda x: re.sub(r'\\_', '', x))

autorun_bad = bad[bad['Type'] == 'Autorun'].copy()
autorun_bad['Info'] = autorun_bad['Info'].apply(lambda x: x.split('--')[1])
autorun_bad['Info'] = autorun_bad['Info'].apply(lambda x: re.sub(r'[\(]No Signed[\)]', '', x))

usb_bad = bad[bad['Type'] == 'USB'].copy()

bad = pd.concat([persistence_bad, process_bad, black_bad, autorun_bad, file_bad, malware_bad], ignore_index=True)
bad['Info'] = bad['Info'].apply(lambda x: x.lower())
bad.drop_duplicates(subset=['MAC_id', 'Info'], keep='first', inplace=True)
bad = pd.concat([bad, usb_bad], ignore_index=True)
bad['label'] = 1

good = data['BinhThuong'].drop(columns=['Thời gian', 'thoigian', 'Chi tiết']).rename(columns={'MoTa': 'Info', 'SuKien': 'Type', 'MAC_Nguon': 'MAC_id'})
persistence_good = good[good['Type'] == 'Persistence'].copy()
persistence_good['Info'] = persistence_good['Info'].apply(lambda x: x.split('ValueData: ')[1].split('--')[0])

file_good = good[good['Type'] == 'Create File'].copy()
file_good['Info'] = file_good['Info'].apply(lambda x: x.split('FileName: ')[1].split('--')[0])

autorun_good = good[good['Type'] == 'Autorun'].copy()
autorun_good['Info'] = autorun_good['Info'].apply(lambda x: x.split('--')[1])
autorun_good['Info'] = autorun_good['Info'].apply(lambda x: re.sub(r'[\(]No Signed[\)]', '', x))

usb_good = good[good['Type'] == 'USB'].copy()

good = pd.concat([persistence_good, file_good, autorun_good], ignore_index=True)
good['Info'] = good['Info'].apply(lambda x: x.lower())
good.drop_duplicates(subset=['MAC_id', 'Info'], keep='first', inplace=True)
good = pd.concat([good, usb_good], ignore_index=True)
good['label'] = 0

final1 = pd.concat([bad, good], ignore_index=True)

data = pd.read_excel('File Path/data_chuoi.xlsx', sheet_name=None, header=None)

bad = data['chuoi_doc'].drop(columns=[2]).rename(columns={0: 'MAC_id', 1: 'Type', 3: 'Info'})

file_bad = bad[bad['Type'] == 'Create File'].copy()
file_bad['Info'] = file_bad['Info'].apply(lambda x: x.split('FileName: ')[1].split('--')[0])

autorun_bad = bad[bad['Type'] == 'Autorun'].copy()
autorun_bad['Info'] = autorun_bad['Info'].apply(lambda x: x.split('--')[1])
autorun_bad['Info'] = autorun_bad['Info'].apply(lambda x: re.sub(r'[\(]No Signed[\)]', '', x))

malware_bad = bad[bad['Type'] == 'Malware'].copy()
malware_bad['Info'] = malware_bad['Info'].apply(lambda x: x.split('--')[1].split(';')[0])
malware_bad['Info'] = malware_bad['Info'].apply(lambda x: re.sub(r'[\(].*[\)]', '', x))
malware_bad['Info'] = malware_bad['Info'].apply(lambda x: re.sub(r'\\\s', '', x))
malware_bad['Info'] = malware_bad['Info'].apply(lambda x: re.sub(r'\\_', '', x))

persistence_bad = bad[bad['Type'] == 'Persistence'].copy()
persistence_bad['Info'] = persistence_bad['Info'].apply(lambda x: x.split('ValueData: ')[1].split('--')[0])
persistence_bad = persistence_bad[persistence_bad['Info'].map(lambda x: len(re.findall(r'\\', x))) != 0]

black_bad = bad[bad['Type'] == 'Black Domain'].copy()
black_bad = black_bad[black_bad['Info'].map(lambda x: len(re.findall(r'\---', x))) != 0]
black_bad['Info'] = black_bad['Info'].apply(lambda x: x.split('---')[1])

usb_bad = bad[bad['Type'] == 'USB'].copy()

bad = pd.concat([persistence_bad, black_bad, autorun_bad, file_bad, malware_bad], ignore_index=True)
bad['Info'] = bad['Info'].apply(lambda x: x.lower())
bad.drop_duplicates(subset=['MAC_id', 'Info'], keep='first', inplace=True)
bad = pd.concat([bad, usb_bad], ignore_index=True)
bad['label'] = 1

good = data['chuoi_sach'].drop(columns=[2]).rename(columns={0: 'MAC_id', 1: 'Type', 3: 'Info'}).drop(index=0)

file_good = good[good['Type'] == 'Create File'].copy()
file_good['Info'] = file_good['Info'].apply(lambda x: x.split('FileName: ')[1].split('--')[0])

autorun_good = good[good['Type'] == 'Autorun'].copy()
autorun_good['Info'] = autorun_good['Info'].apply(lambda x: x.split('--')[1])
autorun_good['Info'] = autorun_good['Info'].apply(lambda x: re.sub(r'[\(]No Signed[\)]', '', x))

policy_good = good[good['Type'] == 'Policy'].copy()
policy_good['Info'] = policy_good['Info'].apply(lambda x: x.split('--')[1].split('. ')[0])

persistence_good = good[good['Type'] == 'Persistence'].copy()
persistence_good['Info'] = persistence_good['Info'].apply(lambda x: x.split('ValueData: ')[1].split('--')[0])
persistence_good = persistence_good[persistence_good['Info'].map(lambda x: len(re.findall(r'\\', x))) != 0]

usb_good = good[good['Type'] == 'USB'].copy()

good = pd.concat([persistence_good, file_good, autorun_good, policy_good], ignore_index=True)
good['Info'] = good['Info'].apply(lambda x: x.lower())
good.drop_duplicates(subset=['MAC_id', 'Info'], keep='first', inplace=True)
good = pd.concat([good, usb_good], ignore_index=True)
good['label'] = 0

final2 = pd.concat([bad, good], ignore_index=True)

final = pd.concat([final1, final2], ignore_index=True)

In [5]:
final['MAC_id'].unique()

329

In [256]:
from gensim.models import FastText, KeyedVectors
import os
def build_word_vectors(train_data, path=None):
    sentences = train_data['Info'].str.split('\\').tolist()
    model = FastText(vector_size=300, window=10, min_count=1, workers=4, sg=0, sentences=sentences)
    # model.build_vocab(sentences=sentences)
    model.train(sentences, total_examples=len(sentences), epochs=10)
    if path is not None:
        model.wv.save(path)
    return model.wv

index = final[(final['Type'] == 'Black Domain') | (final['Type'] == 'Malware') | (final['Type'] == 'USB')].index
data_to_vector = final.drop(index=index)
word_vectors = build_word_vectors(data_to_vector)

In [257]:
data_to_vector['text_tokenized'] = data_to_vector['Info'].str.split('\\')
data_to_vector['text_vect_mean'] = data_to_vector['text_tokenized'].apply(
    lambda x: np.array([word_vectors[token] for token in x]).mean(axis=0))

In [258]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

def get_model(cfg, name='xgb'):
    if name == 'rf':
        model = RandomForestClassifier(
            n_estimators=cfg['n_estimators'], 
            max_depth=cfg['max_depth'], 
            min_samples_split=cfg['min_samples_split'], 
            min_samples_leaf=cfg['min_samples_leaf'], 
            max_features=cfg['max_features'], 
            n_jobs=-1, 
            random_state=42,
            class_weight='balanced'
        )
    elif name == 'dt':
        model = DecisionTreeClassifier(
            max_depth=cfg['max_depth'], 
            min_samples_split=cfg['min_samples_split'], 
            min_samples_leaf=cfg['min_samples_leaf'], 
            max_features=cfg['max_features'],
            random_state=42
        )
    return model

In [3]:
# import pandas as pd
# data = pd.read_csv('full_data1.csv')
# data[data['label'] == 1]['process_path'].values

In [259]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
def score(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return {'accuracy': accuracy, 'precision': precision,
            'recall': recall, 'f1_score': f1}

In [260]:
from sklearn.model_selection import train_test_split
RF_CONFIG = dict(
    n_estimators=60,
    max_depth=34, 
    min_samples_split=2,
    min_samples_leaf=10,
    max_features=0.65
)
data_to_vector = data_to_vector.sample(frac=1.0)
train, test = train_test_split(data_to_vector, test_size=0.1, random_state=42, stratify=data_to_vector['label'])
rf_model = get_model(RF_CONFIG, name='rf')
rf_model.fit(np.stack(train['text_vect_mean']), train['label'])
rf_pred = rf_model.predict(np.stack(test['text_vect_mean']))

In [5]:
# score(test['label'], rf_pred)

In [262]:
data_pred = rf_model.predict(np.stack(data_to_vector['text_vect_mean']))
data_to_vector['event_pred'] = data_pred
data_to_vector.drop(columns=['text_tokenized', 'text_vect_mean'], inplace=True)
mal_df = final[(final['Type'] == 'Black Domain') | (final['Type'] == 'Malware')].copy()
mal_df['event_pred'] = 1
data_final = pd.concat([data_to_vector, mal_df], ignore_index=True)

In [277]:
data_gr = data_final.groupby('MAC_id').agg({'label': ['sum', 'mean'], 'event_pred': ['sum', 'mean'], 'Type': list})
data_gr.columns = ['_'.join(col) for col in data_gr.columns]
data_gr['MAC_label'] = data_gr.apply(lambda x: 1 if ('Malware' in x['Type_list']) or ('Black Domain' in x['Type_list']) else 1 if x['label_mean'] > 0.5 else 0, axis=1)
data_gr['MAC_pred'] = data_gr.apply(lambda x: 1 if ('Malware' in x['Type_list']) or ('Black Domain' in x['Type_list']) else 1 if x['event_pred_mean'] > 0.5 else 0, axis=1)

In [4]:
# score(data_gr['MAC_label'], data_gr['MAC_pred'])